In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 7.9 MB/s eta 0:00:00
Using legacy 'setup.py install' for pyspark, since package 'wheel' is not installed.
  Running setup.py install for pyspark ... done

[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [82]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import datetime
from sqlalchemy import create_engine
import psycopg2
import csv
import glob
import os
import shutil

In [83]:
# initialize spark session
spark = SparkSession.builder.appName("Kumba_construction_Data_Exploration").config("spark.driver.memory", "16gb").getOrCreate()

In [84]:
spark

In [115]:
# Read CSV
con_df = spark.read.csv("kumba_construction_data.csv", header=True, inferSchema=True)

In [116]:
# view schema
con_df.printSchema()

root
 |-- ProjectID: string (nullable = true)
 |-- ProjectName: string (nullable = true)
 |-- ClientName: string (nullable = true)
 |-- ProjectType: string (nullable = true)
 |-- ProjectStatus: string (nullable = true)
 |-- StartDate: string (nullable = true)
 |-- EndDate: string (nullable = true)
 |-- EstimatedBudget: double (nullable = true)
 |-- ActualCost: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- ManagerName: double (nullable = true)
 |-- TeamSize: double (nullable = true)
 |-- ContractorName: double (nullable = true)
 |-- NumberOfSubcontractors: double (nullable = true)
 |-- MaterialsCost: double (nullable = true)
 |-- LaborCost: double (nullable = true)
 |-- EquipmentCost: double (nullable = true)
 |-- PermitFees: double (nullable = true)
 |-- InspectionFees: double (nullable = true)
 |-- ChangeOrderCount: double (nullable = true)
 |-- SafetyIncidentsCount: double (nullable = true)
 |-- ProjectDelayDays: double (nullable = true)
 |-- WeatherDelayDays:

In [117]:
con_df.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+----------+---------------+----------+--------------------+-----------+--------+--------------+----------------------+-------------+---------+-------------+----------+--------------+----------------+--------------------+----------------+----------------+-------------------+-----------------+------------------------+---------------------+---------------+--------------------+----+
|           ProjectID|         ProjectName|          ClientName|         ProjectType|       ProjectStatus| StartDate|   EndDate|EstimatedBudget|ActualCost|            Location|ManagerName|TeamSize|ContractorName|NumberOfSubcontractors|MaterialsCost|LaborCost|EquipmentCost|PermitFees|InspectionFees|ChangeOrderCount|SafetyIncidentsCount|ProjectDelayDays|WeatherDelayDays|ClientFeedbackScore|QualityAuditScore|EnvironmentalImpactScore|EnergyEfficiencyScore|InnovationScore|CommunityImpactScore| ROI|
+-

In [118]:
con_df.describe().show()

+-------+--------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+----------------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+--------------------+-------------------+----------------+-------------------+-----------------+------------------------+---------------------+---------------+--------------------+----+
|summary|           ProjectID|ProjectName|        ClientName|       ProjectType|     ProjectStatus|         StartDate|           EndDate|   EstimatedBudget|        ActualCost|          Location|      ManagerName|          TeamSize|    ContractorName|NumberOfSubcontractors|     MaterialsCost|        LaborCost|    EquipmentCost|       PermitFees|   InspectionFees|  ChangeOrderCount|SafetyIncidentsCount|   ProjectDelayDays|Weather

In [119]:
# check null values
for column in con_df.columns:
    print(column, "Null Values:", con_df.filter(con_df[column].isNull()).count())

ProjectID Null Values: 0


ProjectName Null Values: 0


ClientName Null Values: 0


ProjectType Null Values: 0


ProjectStatus Null Values: 0


...

EnvironmentalImpactScore Null Values: 2200000


EnergyEfficiencyScore Null Values: 2200000


InnovationScore Null Values: 2200000


CommunityImpactScore Null Values: 2200000


ROI Null Values: 2200000


In [91]:
numeric_cols = ["EstimatedBudget", "ActualCost", "ManagerName", "TeamSize", "ContractorName", "NumberOfSubcontractors", 
               "MaterialsCost", "LaborCost", "EquipmentCost", "PermitFees", "InspectionFees", "ChangeOrderCount",
               "SafetyIncidentsCount", "ProjectDelayDays"]

In [120]:
cleaned_data = con_df.fillna(0, subset=numeric_cols)

In [121]:
string_cols = ["ProjectID","ProjectName", "ClientName", "ProjectType", "ProjectStatus", "Location", "WeatherDelayDays", 
               "ClientFeedbackScore", "QualityAuditScore", "EnvironmentalImpactScore", "EnergyEfficiencyScore",
               "InnovationScore", "CommunityImpactScore", "ROI"]

In [122]:
cleaned_data = cleaned_data.fillna("Unknown", subset=string_cols)

In [123]:
# For date columns wwith many nuls, replace 
cleaned_data = cleaned_data.withColumn("StartDate", when(col("StartDate").isNull(), "2022-01-01").otherwise("StartDate"))
cleaned_data = cleaned_data.withColumn("EndDate", when(col("EndDate").isNull(), "2022-12-31").otherwise("EndDate"))

In [124]:
# Implement necessary data conversions
# cleaned_data = cleaned_data.withColumn("ProjectID", cleaned_data["ProjectID"].cast("integer"))
cleaned_data = cleaned_data.withColumn("EstimatedBudget", cleaned_data["EstimatedBudget"].cast("double"))
cleaned_data = cleaned_data.withColumn("ActualCost", cleaned_data["ActualCost"].cast("double"))

In [114]:
cleaned_data


DataFrame[ProjectID: int, ProjectName: string, ClientName: string, ProjectType: string, ProjectStatus: string, StartDate: string, EndDate: string, EstimatedBudget: double, ActualCost: double, Location: string, ManagerName: double, TeamSize: double, ContractorName: double, NumberOfSubcontractors: double, MaterialsCost: double, LaborCost: double, EquipmentCost: double, PermitFees: double, InspectionFees: double, ChangeOrderCount: double, SafetyIncidentsCount: double, ProjectDelayDays: double, WeatherDelayDays: string, ClientFeedbackScore: string, QualityAuditScore: string, EnvironmentalImpactScore: string, EnergyEfficiencyScore: string, InnovationScore: string, CommunityImpactScore: string, ROI: string]

In [113]:
cleaned_data.show()

+---------+--------------------+--------------------+--------------------+--------------------+---------+-------+---------------+----------+--------------------+-----------+--------+--------------+----------------------+-------------+---------+-------------+----------+--------------+----------------+--------------------+----------------+----------------+-------------------+-----------------+------------------------+---------------------+---------------+--------------------+-------+
|ProjectID|         ProjectName|          ClientName|         ProjectType|       ProjectStatus|StartDate|EndDate|EstimatedBudget|ActualCost|            Location|ManagerName|TeamSize|ContractorName|NumberOfSubcontractors|MaterialsCost|LaborCost|EquipmentCost|PermitFees|InspectionFees|ChangeOrderCount|SafetyIncidentsCount|ProjectDelayDays|WeatherDelayDays|ClientFeedbackScore|QualityAuditScore|EnvironmentalImpactScore|EnergyEfficiencyScore|InnovationScore|CommunityImpactScore|    ROI|
+---------+---------------

In [132]:
# file_save_path = r'/Users/isaacoduh/DevSandbox/data_portfolio/DE/pyspark-kumba-case-study/output_data/'
# cleaned_data.write.mode('overwrite').option("header",True) \
#  .csv('/Users/isaacoduh/DevSandbox/data_portfolio/DE/pyspark-kumba-case-study/output_data/')
cleaned_data.coalesce(1).write.mode('overwrite').option("header",True) \
 .csv('/Users/isaacoduh/DevSandbox/data_portfolio/DE/pyspark-kumba-case-study/output_data/')

In [126]:
# Load Data
conn = psycopg2.connect(
    host='127.0.0.1', database='postgres', user='isaacoduh', password='root'
)
cusor = conn.cursor()

In [129]:
# CREATE TABLE
# conn.rollback()
cusor.execute("""
    CREATE TABLE IF NOT EXISTS kumba_construction (
        ProjectID VARCHAR(255),
        ProjectName VARCHAR(255),
        ClientName VARCHAR(255),
        ProjectType VARCHAR(255),
        ProjectStatus VARCHAR(255),
        StartDate DATE,
        EndDate DATE,
        EstimatedBudget FLOAT,
        ActualCost FLOAT,
        Location VARCHAR(255),
        ManagerName FLOAT,
        TeamSize FLOAT,
        ContractorName FLOAT,
        NumberOfSubcontractors FLOAT,
        MaterialsCost FLOAT,
        LaborCost FLOAT,
        EquipmentCost FLOAT,
        PermitFees FLOAT,
        InspectionFees FLOAT,
        ChangeOrderCount FLOAT,
        SafetyIncidentsCount FLOAT,
        ProjectDelayDays FLOAT,
        WeatherDelayDays VARCHAR(255),
        ClientFeedbackScore VARCHAR(255),
        QualityAuditScore VARCHAR(255),
        EnvironmentalImpactScore VARCHAR(255),
        EnergyEfficiencyScore VARCHAR(255),
        InnovationScore VARCHAR(255),
        CommunityImpactScore VARCHAR(255),
        ROI VARCHAR(255)
    )
""")
conn.commit()

In [130]:
# loading layer
# develop a function to connect to pgadmin
def get_db_connection():
    connection = psycopg2.connect(host='127.0.0.1', database='postgres', user='isaacoduh', password='root')
    return connection

# connect to sql database
conn = get_db_connection()

In [131]:
def load_data_from_csv_to_tables(csv_path, table_name):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            placeholders = ", ".join(['%s'] * len(row))
            query = f'INSERT INTO {table_name} VALUES ({placeholders});'
            cursor.execute(query, row)
    conn.commit()
    cursor.close()
    conn.close()
            
        

In [45]:
csv_files = glob.glob('output_data/cleaned_transformed_data_*.csv')

In [134]:
file_path = r'/Users/isaacoduh/DevSandbox/data_portfolio/DE/pyspark-kumba-case-study/output_data/part-00000-179243d9-6ba3-4250-9933-a6cfc7f38404-c000.csv'
load_data_from_csv_to_tables(file_path, 'kumba_construction')
    

InvalidDatetimeFormat: invalid input syntax for type date: "StartDate"
LINE 1: ...rison, Evans and Ward', 'Commercial', 'Planning', 'StartDate...
                                                             ^
